In [20]:
import pandas as pd
from pymongo import MongoClient
import requests
import json

In [21]:
client = MongoClient('mongodb://localhost:27017')
db = client.mydatabase
collection = db.mycollection

In [22]:
API_KEY = 'HMU3C6GSU9RRXY5Z'
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey={API_KEY}'
r = requests.get(url)
data = r.json()
collection.insert_one(data['Time Series (Daily)'])

In [23]:
# create a MongoDB client instance
# client = MongoClient()

# get a reference to the database
db = client.mydatabase

# get a reference to the collection
collection = db.closing_prices

# define your Alpha Vantage API key and endpoint URL
api_key = "YOUR_API_KEY"
url = "https://www.alphavantage.co/query"

# define the symbol and data type to retrieve
symbol = "AAPL"
data_type = "TIME_SERIES_DAILY_ADJUSTED"

# define the API parameters
params = {
    "function": data_type,
    "symbol": symbol,
    "apikey": api_key
}

# make a request to the Alpha Vantage API
response = requests.get(url, params=params)

# convert the response data to a Python dictionary
data = json.loads(response.text)

# insert each day's closing price as a separate document
for date, values in data["Time Series (Daily)"].items():
    document = {
        "symbol": symbol,
        "timestamp": pd.to_datetime(date),
        "close": float(values["4. close"])
    }
    collection.insert_one(document)
